In [1]:
!pip install streamlit
!pip install pyngrok
!pip install tensorflow

In [62]:
%%writefile app.py
import numpy as np
from keras.utils import get_file
import tensorflow as tf
from tensorflow import keras
from keras.optimizers import SGD
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules
from tensorflow.keras.applications import vgg19
from keras.utils import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import vgg19
from keras.utils import plot_model
from keras import Model
import streamlit as st

st.set_page_config(page_title="Neural Style Transfer")
st.title('Welcome to Neural Style Transfer')

st.subheader("Choose Images")
# Loading of images
style = st.file_uploader("Select Style Image", type=["png","jpg","jpeg"], key="style_img")
content = st.file_uploader("Select Content Image", type=["png","jpg","jpeg"], key="content_img")

if style is not None:
    # To See details
    file_details = {"filename":style.name, "filetype":style.type, "filesize":style.size}
    # st.write(file_details)

    # To View Uploaded Image
    st.write("Style Image")
    st.image(style,width=500)

    with open(style.name,"wb") as f:
        f.write((style).getbuffer())
    style_image_path = style.name


if content is not None:
    # To See details
    file_details = {"filename":content.name, "filetype":content.type, "filesize":content.size}
    # st.write(file_details)

    # To View Uploaded Image
    st.write("Content Image")
    st.image(content,width=500)

    with open(content.name,"wb") as f:
        f.write((content).getbuffer())
    base_image_path = content.name

hide_img_fs = '''
<style>
button[title="View fullscreen"]{
    visibility: hidden;}
</style>
'''

def preprocess_image(image_path):
    # Util function to open, resize and format pictures into appropriate tensors
    img =load_img(
        image_path, target_size=(img_nrows, img_ncols)
    )
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return tf.convert_to_tensor(img)

def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram

def content_loss(base, combination):
    return tf.reduce_sum(tf.square(combination - base))

model = vgg19.VGG19(weights="imagenet", include_top=False)

outputs_dict= dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = Model(inputs=model.inputs, outputs=outputs_dict)

style_cnn = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]

content_cnn = "block5_conv2"

content_weight = 2.5e-8
style_weight = 1e-6

def loss_function(combination_image, base_image, style_reference_image):

    # 1. Combine all the images in the same tensioner.
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0
    )

    # 2. Get the values in all the layers for the three images.
    features = feature_extractor(input_tensor)

    #3. Inicializar the loss

    loss = tf.zeros(shape=())

    # 4. Extract the content layers + content loss
    layer_features = features[content_cnn]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]

    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )
    # 5. Extraer the style layers + style loss
    for layer_name in style_cnn:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        sl = style_loss(style_reference_features, combination_features)
        loss += (style_weight / len(style_cnn)) * sl

    return loss

@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = loss_function(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

def result_saver(i):
  # Create name
  image_name = 'At_'+ str(i) + '_Iteration.png'

  # Save image
  img = deprocess_image(combination_image.numpy())
  keras.preprocessing.image.save_img(image_name, img)

optimizer = SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
    )
)


def run_style_transfer(iteration):
  for i in range(1, iterations + 1):
      loss, grads = compute_loss_and_grads(
          combination_image, base_image, style_reference_image
      )
      optimizer.apply_gradients([(grads, combination_image)])
      if i % 10 == 0:
          print("Iteration %d: loss=%.2f" % (i, loss))
          result_saver(i)
          scaled_value = ((i - 1) / (iterations - 1)) * (100 - 0) + 0
          my_bar.progress(int(scaled_value), text=progress_text)
  return True

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

def deprocess_image(x):

    # Convert the tensor into Array
    x = x.reshape((img_nrows, img_ncols, 3))

    # We ensure that they do not have an average of 0
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68

    # convert from BGR to RGB
    x = x[:, :, ::-1]

    # Ensure that they are between 0 and 255
    x = np.clip(x, 0, 255).astype("uint8")

    return x

st.markdown(hide_img_fs, unsafe_allow_html=True)

if style is not None and content is not None:
    img_nrows = 400
    width, height = load_img(base_image_path).size
    img_ncols = int(width * img_nrows / height)
    base_image = preprocess_image(base_image_path)
    style_reference_image = preprocess_image(style_image_path)
    combination_image = tf.Variable(preprocess_image(base_image_path))
    iterations = st.slider('Select the iterations?', 0, 400, 50, 20)
    if st.button("Run"):
        progress_text = "Operation in progress. Please wait."
        my_bar = st.progress(0, text=progress_text)
        val = run_style_transfer(iterations)
        if val:
            st.write("Output Image at 10 Iteration")
            st.image("At_10_Iteration.png",width=500)
            st.write("Output Image at "+str(iterations)+" Iteration")
            st.image("At_"+str(iterations)+"_Iteration.png",width=500)

Overwriting app.py


In [3]:
from pyngrok import ngrok
ngrok.kill()
ngrok.set_auth_token("Your Token")
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect('80')
print(url)

nohup: appending output to 'nohup.out'


NgrokTunnel: "https://ec74-34-126-177-84.ngrok-free.app" -> "http://localhost:80"
